<a href="https://colab.research.google.com/github/vaishu7989/DATAFLOW-colab/blob/main/sample_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!{'pip install --quiet apache_beam'}

In [11]:
!{'mkdir -p vaishu_colab'}

In [12]:
!ls

dept_data.txt  sample_data  vaishu_colab


In [4]:
from google.colab import files
uploaded = files.upload()

Saving dept_data.txt to dept_data.txt


In [5]:
!ls

dept_data.txt  sample_data  vaishu_colab


In [ ]:
!cat dept_data.txt

In [ ]:
#creating pcollection from your data

In [14]:
import apache_beam as beam

def SplitRow(element):
  return element.split(',')
def filtering(record):
  return record[3] == 'Accounts'

p1 = beam.Pipeline()

attendence_count = (
    p1
    |beam.io.ReadFromText('dept_data.txt',strip_trailing_newlines = True)
    |beam.Map(SplitRow)
    |beam.Filter(filtering)
    |beam.Map(lambda record: (record[1],1))
    #lambda function is the small anonymous fun which can take no.of arguments and gives single argument.
    |beam.CombinePerKey(sum)
    # CombinePerkey = groupby key + combiner + reducer in argument we can pass some built in functions and as per our requirement.


    |beam.io.WriteToText('vaishu_colab/sample_run')
)
p1.run()

#sample the first 20 results,remember there are no ordering guarantees.
!{('head -n 20 vaishu_colab/sample_run-00000-of-00001')}

('Marco', 31)
('Rebekah', 31)
('Itoe', 31)
('Edouard', 31)
('Kyle', 62)
('Kumiko', 31)
('Gaston', 31)
('Ayumi', 30)


In [15]:
!ls vaishu_colab

sample_run-00000-of-00001


In [16]:
!cat vaishu_colab/sample_run-00000-of-00001

('Marco', 31)
('Rebekah', 31)
('Itoe', 31)
('Edouard', 31)
('Kyle', 62)
('Kumiko', 31)
('Gaston', 31)
('Ayumi', 30)


In [ ]:
# Labeling Transformations providing unique label

In [17]:
import apache_beam as beam
p2 = beam.Pipeline()

ATT_CNT = (
    p2
    |'Read from file' >> beam.io.ReadFromText('dept_data.txt')
    |'Map transform' >> beam.Map(lambda record: record.split(','))
    |'Filter transform' >> beam.Filter(lambda record: record[3] == 'Accounts')
    |'Transform' >> beam.Map(lambda record: (record[1],1))
    |'Combineperkey transform' >> beam.CombinePerKey(sum)
    |'Write into file' >> beam.io.WriteToText('vaishu_colab/sample_run_label')
)

p2.run()

!{('head -n 20 vaishu_colab/sample_run_label-00000-of-00001')}

('Marco', 31)
('Rebekah', 31)
('Itoe', 31)
('Edouard', 31)
('Kyle', 62)
('Kumiko', 31)
('Gaston', 31)
('Ayumi', 30)


In [18]:
import apache_beam as beam

with beam.Pipeline() as p3:
  count_attendence = (
      p3
      |'Read from file' >> beam.io.ReadFromText('dept_data.txt')
      |'Map transform' >> beam.Map(lambda record: record.split(','))
      |'Filter transform' >> beam.Filter(lambda record: record[3] == 'Accounts')
      |'Transform' >> beam.Map(lambda record: (record[1],1))
      |'Combineperkey transform' >> beam.CombinePerKey(sum)
      |'Write into file' >> beam.io.WriteToText('vaishu_colab/sample_run_pipeline_name')
)

# p3.run()

!{('head -n 20 vaishu_colab/sample_run_pipeline_name-00000-of-00001')}

('Marco', 31)
('Rebekah', 31)
('Itoe', 31)
('Edouard', 31)
('Kyle', 62)
('Kumiko', 31)
('Gaston', 31)
('Ayumi', 30)


In [ ]:
#Applying same pcollection as input for multiple transforms(branched pipeline)

In [30]:
import apache_beam as beam

def SplitRow(element):
  return element.split(',')

p = beam.Pipeline()

input_collection = (
    p
    |"read from text file" >> beam.io.ReadFromText('dept_data.txt')
    |"Split rows" >> beam.Map(SplitRow)

)
accounts_count = (
    input_collection
    |'Get all Accounts dept persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
    |'pair each accounts employee with 1' >> beam.Map(lambda record: ("Account, " +record[1],1))
    |'Group and sum1' >> beam.CombinePerKey(sum)
    # |'Write results for account' >> beam.io.WriteToText('vaishu_colab/Account')
)

hr_count = (
    input_collection
    | 'Get all HR dept persons' >> beam.Filter(lambda record: record[3] == 'HR')
    | 'pair each hr employee with 1' >> beam.Map(lambda record: ("HR, " +record[1],1))
    | 'Group and sum' >> beam.CombinePerKey(sum)
    # | 'Write results for hr' >> beam.io.WriteToText('vaishu_colab/HR')

)

#new pcollection
output = (
    (accounts_count,hr_count)
    |beam.Flatten()
    |beam.io.WriteToText('vaishu_colab/both')
)

p.run()

!{('head -n 20 vaishu_colab/both-00000-of-00001')}

# !{('head -n 20 vaishu_colab/Account-00000-of-00001')}

# !{('head -n 20 vaishu_colab/HR-00000-of-00001')}




('Account, Marco', 31)
('Account, Rebekah', 31)
('Account, Itoe', 31)
('Account, Edouard', 31)
('Account, Kyle', 62)
('Account, Kumiko', 31)
('Account, Gaston', 31)
('Account, Ayumi', 30)
('HR, Beryl', 62)
('HR, Olga', 31)
('HR, Leslie', 31)
('HR, Mindy', 31)
('HR, Vicky', 31)
('HR, Richard', 31)
('HR, Kirk', 31)
('HR, Kaori', 31)
('HR, Oscar', 31)


In [ ]:
#DoFn is a beam class that defines a distributed processing function.
#it contains all the logic to run the user provided function parallelly on different machines.

In [29]:
import apache_beam as beam

class SplitRow(beam.DoFn):

  def process(self, element):
    # return type -> list
    return  [element.split(',')]


class FilterAccountsEmployee(beam.DoFn):

  def process(self, element):
    if element[3] == 'Accounts':
      return [element]

class PairEmployees(beam.DoFn):

  def process(self, element):
    return [(element[3]+","+element[1], 1)]

class Counting(beam.DoFn):

  def process(self, element):
    # return type -> list
    (key, values) = element           # [Marco, Accounts  [1,1,1,1....] , Rebekah, Accounts [1,1,1,1,....] ]
    return [(key, sum(values))]


p1 = beam.Pipeline()

attendance_count = (

   p1
    |beam.io.ReadFromText('dept_data.txt')

    |beam.ParDo(SplitRow())
   # | 'Compute WordLength' >> beam.ParDo(lambda element: [ element.split(',') ])

    |beam.ParDo(FilterAccountsEmployee())
    |beam.ParDo(PairEmployees())
    | 'Group ' >> beam.GroupByKey()
    | 'Sum using ParDo' >> beam.ParDo(Counting())

    |beam.io.WriteToText('vaishu_colab/Pardo_trail')

)

p1.run()

# Sample the first 20 results, remember there are no ordering guarantees.
!{('head -n 20 vaishu_colab/Pardo_trail-00000-of-00001')}

('Accounts,Marco', 31)
('Accounts,Rebekah', 31)
('Accounts,Itoe', 31)
('Accounts,Edouard', 31)
('Accounts,Kyle', 62)
('Accounts,Kumiko', 31)
('Accounts,Gaston', 31)
('Accounts,Ayumi', 30)


In [ ]:
#Combiner is a mini reducer which does the reduce task locally to a mapper machine but here in beam it is more than that.

In [31]:
import apache_beam as beam

p = beam.Pipeline()

class AverageFn(beam.CombineFn):

  def create_accumulator(self):
     return (0.0, 0)   # initialize (sum, count)

  def add_input(self, sum_count, input):
    (sum, count) = sum_count
    return sum + input, count + 1

  def merge_accumulators(self, accumulators):

    ind_sums, ind_counts = zip(*accumulators)       # zip - [(27, 3), (39, 3), (18, 2)]  -->   [(27,39,18), (3,3,2)]
    return sum(ind_sums), sum(ind_counts)        # (84,8)

  def extract_output(self, sum_count):

    (sum, count) = sum_count    # combine globally using CombineFn
    return sum / count if count else float('NaN')


small_sum = (
           p
            | beam.Create([15,5,7,7,9,23,13,5])
            | "Combine Globally" >> beam.CombineGlobally(AverageFn())
            | 'Write results' >> beam.io.WriteToText('vaishu_colab/combine')
          )
p.run()

# Sample the first 20 results, remember there are no ordering guarantees.
!{'head -n 20 vaishu_colab/combine-00000-of-00001'}

10.5


In [34]:
import apache_beam as beam

class MyTransform(beam.PTransform):

  def expand(self, input_coll):

    a = (
        input_coll
                       | 'Group and sum1' >> beam.CombinePerKey(sum)
                       | 'count filter accounts' >> beam.Filter(filter_on_count)
                       | 'Regular accounts employee' >> beam.Map(format_output)

    )
    return a

def SplitRow(element):
    return element.split(',')


def filter_on_count(element):
  name, count = element
  if count > 30:
    return element

def format_output(element):
  name, count = element
  return ', '.join((name,str(count),'Regular employee'))

p = beam.Pipeline()

input_collection = (
                      p
                      | "Read from text file" >> beam.io.ReadFromText('dept_data.txt')
                      | "Split rows" >> beam.Map(SplitRow)
                   )

accounts_count = (
                      input_collection
                      | 'Get all Accounts dept persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
                      | 'Pair each accounts employee with 1' >> beam.Map(lambda record: ("Accounts, " +record[1], 1))
                      | 'composite accoubts' >> MyTransform()
                      | 'Write results for account' >> beam.io.WriteToText('vaishu_colab/Account')
                 )

hr_count = (
                input_collection
                | 'Get all HR dept persons' >> beam.Filter(lambda record: record[3] == 'HR')
                | 'Pair each hr employee with 1' >> beam.Map(lambda record: ("HR, " +record[1], 1))
                | 'composite HR' >> MyTransform()
                | 'Write results for hr' >> beam.io.WriteToText('vaishu_colab/HR')
           )
p.run()

# Sample the first 20 results, remember there are no ordering guarantees.
!{('head -n 20 vaishu_colab/Account-00000-of-00001')}
!{('head -n 20 vaishu_colab/HR-00000-of-00001')}

Accounts, Marco, 31, Regular employee
Accounts, Rebekah, 31, Regular employee
Accounts, Itoe, 31, Regular employee
Accounts, Edouard, 31, Regular employee
Accounts, Kyle, 62, Regular employee
Accounts, Kumiko, 31, Regular employee
Accounts, Gaston, 31, Regular employee
HR, Beryl, 62, Regular employee
HR, Olga, 31, Regular employee
HR, Leslie, 31, Regular employee
HR, Mindy, 31, Regular employee
HR, Vicky, 31, Regular employee
HR, Richard, 31, Regular employee
HR, Kirk, 31, Regular employee
HR, Kaori, 31, Regular employee
HR, Oscar, 31, Regular employee


In [35]:
#CogroupBykey
#Relational join of two or more key/value Pcollections.
#Accepts a dictionary of key/value Pcollections and output a single Pcollection containing 1 key/value tuple for each key in the input Pcollections.